<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#This-notebook-will-not-run-yet!!!" data-toc-modified-id="This-notebook-will-not-run-yet!!!-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>This notebook will not run yet!!!</a></span></li></ul></div>

# This notebook will not run yet!!!
Dependencies: optical flow computed over bounding boxes, object detections.

In [ ]:
import scannerpy
import scannertools as st
from query.models import Video, Object
from esper.prelude import *
import rekall as rktmux
from rekall.interval_list import IntervalList
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.payload_predicates import payload_satisfies
from esper.rekall import intrvllists_to_result, intrvllists_to_result_with_objects
import numpy as np

In [ ]:
video_id=123
video = Video.objects.get(id=123)

In [ ]:
# Load objects
objects = Object.objects.filter(frame__video_id=video_id).annotate(
    frame_num=F('frame__number'),
    video_id=F('frame__video_id'))
obj_collection = VideoIntervalCollection.from_django_qs(
    objects,
    schema = {
        'start': 'frame_num',
        'end': 'frame_num'
    },
    with_payload=in_array(bbox_payload_parser(VideoIntervalCollection.django_accessor))
).coalesce(payload_merge_op=payload_plus)

# Prepare list of frames and bounding boxes for Scanner
frames = [[intrvl.get_start() for intrvl in obj_collection.get_intervallist(video).get_intervals()]
          for video in obj_collection.get_allintervals()]
bboxes = [[[{'bbox_x1': bbox['bbox_x1'],
            'bbox_y1': bbox['bbox_y1'],
            'bbox_x2': bbox['bbox_x2'],
            'bbox_y2': bbox['bbox_y2']} for bbox in intrvl.get_payload()]
            for intrvl in obj_collection.get_intervallist(video).get_intervals()]
          for video in obj_collection.get_allintervals()]

In [ ]:
# Compute flow histograms over bounding boxes in Scanner
db = scannerpy.Database()

flow_histogram_bboxes = st.histograms.compute_flow_histograms_bboxes(
    db,
    videos=[video.for_scannertools()],
    frames=frames,
    bboxes=bboxes)

In [ ]:
# Annotate object payloads with flow histogram results
for video_id, framelist, flows_in_video in zip([video_id], frames, flow_histogram_bboxes):
    intervallist = objects.get_intervallist(video_id)
    for frame, flows_in_frame, interval in zip(framelist, flows_in_video.load(), intervallist.get_intervals()):
        for obj, flow in zip(interval.payload, flow):
            obj['flow_hist'] = flow

In [ ]:
# Filter out all objects whose flow histograms don't have a large amount of flow
def significant_flow(flow_hist):
    avg = video.width * video.height / 64
    return np.max(hist[1][1:] > 10 * avg)
objects = objects.map(
    lambda intrvl: (intrvl.start, intrvl.end,
                   [obj for obj in intrvl.payload
                    if significant_flow(obj['flow_hist'])]))
objects = objects.filter(payload_satisfies(length_at_least(1)))

In [ ]:
# Display objects with significant motion flow
esper_widget(intrvllists_to_result_bbox(objects), show_middle_frame=False)